In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
cd gdrive/My Drive/"Game bot detection"/"final data & analysis"

/content/gdrive/My Drive/Game bot detection/final data & analysis


In [0]:
import numpy as np
import pandas as pd

In [0]:
dataset_with_perday = pd.read_csv('dataset_with_perday.csv', engine='python')
data_perday_alpha = pd.read_csv('data_perday_alpha.csv', engine='python')
#fill na
dataset_with_perday.fillna(0, inplace = True)
# There are inf values. Change it to max value in the column
for i in dataset_with_perday.columns:
    dataset_with_perday.loc[dataset_with_perday[i] == np.inf, i] = np.nan
    dataset_with_perday.loc[np.isnan(dataset_with_perday[i]), i] = max(dataset_with_perday[i])

for i in data_perday_alpha.columns:
    data_perday_alpha.loc[data_perday_alpha[i] == np.inf, i] = np.nan
    data_perday_alpha.loc[np.isnan(data_perday_alpha[i]), i] = max(data_perday_alpha[i])

del data_perday_alpha['Unnamed: 0']
data_perday_alpha.head()

,login_count,play_time,max_level,playtime_per_day,abyss,sit_count,exp_get_amout,money_get_count,teleport_count,killed_by_pc,killed_by_npc,reborn_count,login_total_day,sit_count_perday,login_total_day.1,item_get_count_perday,money_get_count_perday,use_portal_count_perday,teleport_count_perday,total_party_time,sit_count/play_time,killed_by_pc/play_time,killed_by_npc/play_time,total-party_time/play_time,teleport_count/play_time,play_time/login_count,exp_get_amount/play_time,playtime_per_day/login_total_day,max_level/login_count,reborn_count/login_count,sit_count/max_level,play_time/question_count,class
0,259.0,637926.0,0.0,42528.40000,0.0,270.0,3.891496e+08,4686.0,376.0,47.0,30.0,51.0,15.0,18.000000,15.0,2076.600000,312.400000,0.000000,25.066667,8862.0,0.000423,0.000074,0.000047,0.013892,0.000589,2463.034749,610.023075,2835.226667,0.000000,0.196911,0.000000,10289.129032,0.0
1,38.0,401885.0,42.0,28706.07143,0.0,153.0,8.011943e+07,1596.0,343.0,32.0,75.0,75.0,14.0,10.928571,14.0,4398.500000,114.000000,0.000000,24.500000,69458.0,0.000381,0.000080,0.000187,0.172831,0.000853,10575.921053,199.359088,2050.433674,1.105263,1.973684,3.642857,68.197013,0.0
2,43.0,240334.0,38.0,17166.71429,0.0,378.0,1.424854e+08,1318.0,337.0,35.0,53.0,69.0,14.0,27.000000,14.0,1763.285714,94.142857,0.000000,24.071429,43619.0,0.001573,0.000146,0.000221,0.181493,0.001402,5589.162791,592.864002,1226.193878,0.883721,1.604651,9.947368,2477.670103,0.0
3,25.0,1205493.0,0.0,80366.20000,0.0,207.0,1.377553e+09,7979.0,44.0,0.0,12.0,10.0,15.0,13.800000,15.0,7319.666667,531.933333,0.000000,2.933333,0.0,0.000172,0.000000,0.000010,0.000000,0.000036,48219.720000,1142.729893,5357.746667,0.000000,0.400000,0.000000,0.000000,0.0
4,110.0,457433.0,0.0,30495.53333,0.0,320.0,3.950991e+08,3551.0,568.0,311.0,109.0,261.0,15.0,21.333333,15.0,6557.866667,236.733333,0.066667,37.866667,40934.0,0.000700,0.000680,0.000238,0.089486,0.001242,4158.481818,863.731014,2033.035555,0.000000,2.372727,0.000000,5717.912500,0.0


# 모델 학습

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import xgboost as xgb

In [0]:
model_data = dataset_with_perday.copy()
#del model_data['actor_account']

x = model_data.iloc[:, :-1]
y = model_data.iloc[:, -1]

In [0]:
model_data.head()

,actor_account,login_count,logout_count,login_day_count,play_time,avg_money,ip_count,max_level,login_total_day_x,playtime_per_day,sit_count,exp_get_amout,item_get_count,exp_repair_count,money_get_count,abyss,use_portal_count,killed_by_pc,killed_by_npc,teleport_count,reborn_count,question_count,login_total_day_y,sit_count_perday,item_get_count_perday,exp_repair_count_perday,money_get_count_perday,use_portal_count_perday,teleport_count_perday,total_party_time,guild_join_count,average_party_time,class
0,20000005.0,259.0,259.0,1.0,637926.0,124.849465,259.0,0.0,15.0,42528.40000,270.0,3.891496e+08,31149.0,0.0,4686.0,0.0,0.0,47.0,30.0,376.0,51.0,62.0,15.0,18.000000,2076.600000,0.0,312.400000,0.000000,25.066667,8862.0,0.0,1477.000000,0.0
1,20000016.0,38.0,37.0,1.0,401885.0,43.509962,38.0,42.0,14.0,28706.07143,153.0,8.011943e+07,61579.0,0.0,1596.0,0.0,0.0,32.0,75.0,343.0,75.0,5893.0,14.0,10.928571,4398.500000,0.0,114.000000,0.000000,24.500000,69458.0,0.0,3655.684211,0.0
2,20000033.0,43.0,42.0,1.0,240334.0,-6.613812,43.0,38.0,14.0,17166.71429,378.0,1.424854e+08,24686.0,0.0,1318.0,0.0,0.0,35.0,53.0,337.0,69.0,97.0,14.0,27.000000,1763.285714,0.0,94.142857,0.000000,24.071429,43619.0,0.0,2295.736842,0.0
3,20000035.0,25.0,24.0,1.0,1205493.0,0.088531,25.0,0.0,15.0,80366.20000,207.0,1.377553e+09,109795.0,0.0,7979.0,0.0,0.0,0.0,12.0,44.0,10.0,0.0,15.0,13.800000,7319.666667,0.0,531.933333,0.000000,2.933333,0.0,0.0,0.000000,0.0
4,20000051.0,110.0,109.0,1.0,457433.0,-32.874878,110.0,0.0,15.0,30495.53333,320.0,3.950991e+08,98368.0,0.0,3551.0,0.0,1.0,311.0,109.0,568.0,261.0,80.0,15.0,21.333333,6557.866667,0.0,236.733333,0.066667,37.866667,40934.0,0.0,2558.375000,0.0


In [0]:
print(list(y).count(1))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

600


## Smote & min,max scaling

In [0]:
from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler.fit_transform(x_train)
# x_train = scaler.fit_transform(x_train)

from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

# 모델설정
sm = SMOTE(ratio='auto', kind='regular')

# train데이터를 넣어 복제함
X_resampled, Y_resampled = sm.fit_sample(x_train,list(y_train))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


## 랜덤 포레스트

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
rnd_clf = RandomForestClassifier(
  bootstrap = True,
  max_depth = None,
  max_features= 'auto',
  max_leaf_nodes = None,
  min_impurity_decrease = 0.0,
  min_impurity_split = None,
  min_samples_leaf = 1,
  min_samples_split = 2,
  min_weight_fraction_leaf = 0.0,
  n_estimators = 2000,
  n_jobs = 1,
  oob_score = False,
  random_state = 42,
  verbose = 0,
  warm_start = False)
#rnd_clf.fit(x_train, y_train) #smote 처리 안한 것
rnd_clf.fit(X_resampled, Y_resampled) #smote 처리 한 것
y_pred_rf = rnd_clf.predict(x_test)

In [0]:
from sklearn.metrics import accuracy_score, f1_score
print("Accuracy: %.2f" %accuracy_score(y_test, y_pred_rf))
print("F1 score: %.2f" %f1_score(y_test, y_pred_rf))

IndexError: ignored

### Random forest grid search

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [0]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring='f1')
# Fit the random search model
rf_random.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 22.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 40.3min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [0]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': 50,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [0]:
rnd_clf = RandomForestClassifier(
  bootstrap = False,
  max_depth = 50,
  max_features= 'auto',
  min_samples_leaf = 1,
  min_samples_split = 2,
  n_estimators = 1000)

#rnd_clf.fit(x_train, y_train) #smote 처리 안한 것
rnd_clf.fit(X_resampled, Y_resampled) #smote 처리 한 것
y_pred_rf = rnd_clf.predict(x_test)
from sklearn.metrics import accuracy_score, f1_score
print("Accuracy: %.2f" %accuracy_score(y_test, y_pred_rf))
print("F1 score: %.2f" %f1_score(y_test, y_pred_rf))





Accuracy: 0.98
F1 score: 0.82


## 로지스틱

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(estimators=[("lr", log_clf),("rf", rnd_clf),("svc", svm_clf)], voting="hard")

#log_clf.fit(x_train, y_train)
log_clf.fit(X_resampled, Y_resampled)
#y_pred = log_clf.predict(x_test)
y_pred = np.where(log_clf.predict_proba(x_test)[:,1]>0.8, 1, 0) #확률을 조정하는 부분

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
print(list(y_pred).count(1))

234


In [0]:
from sklearn.metrics import accuracy_score, f1_score
print("Accuracy: %.2f" %accuracy_score(y_test, y_pred))
print("F1 score: %.2f" %f1_score(y_test, y_pred))

Accuracy: 0.94
F1 score: 0.64


## Xgboost

In [0]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(n_estimators=2000,
                        n_jobs=4,
                        max_depth=15,
                        learning_rate=0.001,
                        gamma = 0.02,
                        subsample = 0.9,
                        colsample_bytree=0.9,
                        missing=-999,
                        tree_method='gpu_hist'
                        )
xgb_model.fit(X_resampled, Y_resampled)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0.02,
              learning_rate=0.001, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=-999, n_estimators=2000, n_jobs=4,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, tree_method='gpu_hist', verbosity=1)

In [0]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)
xgb_model = xgb.XGBClassifier()
param_grid = { 
    'n_estimators': [200, 500, 1000, 2000],
    'learning_rate': [0.1, 0.01, 0.05],
    'max_depth' : [4,6,8,10],
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}
CV_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv= 5)
CV_xgb.fit(x_train, y_train)
CV_xgb.best_params_

KeyboardInterrupt: ignored

# csv 파일로 만들기

In [0]:
Player_information.to_csv("Player_information.csv", encoding="utf8")
Player_action.to_csv("Player_action.csv", encoding="utf8")
Group_activity.to_csv("Group_activity.csv", encoding="utf8")